In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train_ctrUa4K.csv')
train.head(20)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [3]:
#Converting categorical data to numbers
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [4]:
#Checking for missing values
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
train.shape

(614, 13)

In [6]:
#Dropping the missing values
train = train.dropna()

In [7]:
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
train.shape

(480, 13)

In [9]:
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount','Credit_History']]
y = train.Loan_Status
X.shape, y.shape

((480, 5), (480,))

In [10]:
#Model building

from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size = 0.2,
random_state = 10)

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=4, random_state = 10)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=4, random_state=10)

In [12]:
from sklearn.metrics import accuracy_score
pred_cv = model.predict(x_cv)
accuracy_score(y_cv,pred_cv)

0.8020833333333334

In [13]:
pred_train = model.predict(x_train)
accuracy_score(y_train,pred_train)

0.8203125

In [14]:
# saving the model
import pickle
pickle_out = open("classifier.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [15]:
!pip install -q pyngrok

In [16]:
!pip install -q streamlit

In [17]:
!pip install -q streamlit_ace

In [18]:
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open('classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)
 
@st.cache()
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History):   
 
    # Pre-processing user input    
    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1
 
    if Married == "Unmarried":
        Married = 0
    else:
        Married = 1
 
    if Credit_History == "Unclear Debts":
        Credit_History = 0
    else:
        Credit_History = 1  
 
    LoanAmount = LoanAmount / 1000
 
    # Making predictions 
    prediction = classifier.predict( 
        [[Gender, Married, ApplicantIncome, LoanAmount, Credit_History]])
     
    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred
      
  
# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female"))
    Married = st.selectbox('Marital Status',("Unmarried","Married")) 
    ApplicantIncome = st.number_input("Applicants monthly income") 
    LoanAmount = st.number_input("Total loan amount")
    Credit_History = st.selectbox('Credit_History',("Unclear Debts","No Unclear Debts"))
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History) 
        st.success('Your loan is {}'.format(result))
        print(LoanAmount)
     
if __name__=='__main__': 
    main()

2021-06-21 15:58:26.959 
  command:

    streamlit run C:\Users\Mr.Henry Otoo-Mensah\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [25]:
from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

2021-06-21 16:05:40.458 Opening tunnel named: http-8501-23b1c8de-dcf3-4431-868e-71dc3acadd42


2021-06-21 16:07:29.582 t=2021-06-21T16:07:29+0100 lvl=info msg="no configuration paths supplied"
2021-06-21 16:07:29.582 t=2021-06-21T16:07:29+0100 lvl=info msg="using configuration at default config path" path="C:\\Users\\Mr.Henry Otoo-Mensah/.ngrok2/ngrok.yml"
2021-06-21 16:07:29.582 t=2021-06-21T16:07:29+0100 lvl=info msg="open config file" path="C:\\Users\\Mr.Henry Otoo-Mensah\\.ngrok2\\ngrok.yml" err=nil
2021-06-21 16:07:29.597 t=2021-06-21T16:07:29+0100 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-06-21 16:07:30.915 t=2021-06-21T16:07:30+0100 lvl=info msg="tunnel session started" obj=tunnels.session
2021-06-21 16:07:30.925 t=2021-06-21T16:07:30+0100 lvl=info msg="client session established" obj=csess id=a863d77e3f31
2021-06-21 16:07:30.948 t=2021-06-21T16:07:30+0100 lvl=info msg=start pg=/api/tunnels id=43370ba27148ff58
2021-06-21 16:07:30.953 t=2021-06-21T16:07:30+0100 lvl=info msg=end pg=/api/tunnels id=43370ba27148ff58 status=200 dur=1.2391ms
2021-06-2

<NgrokTunnel: "http://b3d35a441805.ngrok.io" -> "http://localhost:8501">

2021-06-21 16:07:31.543 t=2021-06-21T16:07:31+0100 lvl=info msg=end pg="/api/tunnels/http-8501-23b1c8de-dcf3-4431-868e-71dc3acadd42 (http)" id=0ad49d238d4a9c59 status=200 dur=0s
